In [1]:
%pip install plotly

import plotly.express as px
import pandas as pd
import re
import seaborn as sns
import requests
from tqdm import tqdm
import sqlite3

Note: you may need to restart the kernel to use updated packages.


## Load the data

### Health

In [2]:
#Cargar los datos del csv health
df_hnp = pd.read_csv('HNP_StatsData Alfonso.csv')

### CO2

In [3]:
#Cargar los datos del csv co2
df_co2 = pd.read_csv('owid-co2-data.csv')

## Data preprocessing

In [4]:
df_co2.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [5]:
df_hnp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,141.457567,141.603817,141.796749,141.651778,141.595374,141.593273,...,103.629032,101.905042,100.133826,98.367869,96.574004,95.011793,93.43222,91.845198,NaN,NaN
1,Africa Eastern and Southern,AFE,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Africa Eastern and Southern,AFE,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Eastern and Southern,AFE,Adults (ages 15+) living with HIV,SH.DYN.AIDS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Eastern and Southern,AFE,Adults (ages 15-49) newly infected with HIV,SH.HIV.INCD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Hago una primera limpieza del dataframe: elimino la columna 66, cambio el formato de long a tall.
#Convierto las columnas en filas con el método melt. Primer parámetro las columnas que quiero mantener, y toma el resto por defecto para pasarlas a filas, renombro esa variable con year.
df_hnp = df_hnp.drop(columns='Unnamed: 66')\
    .melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])\
    .rename({'variable': 'Year'}, axis=1)


In [7]:
df_hnp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,value
0,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,141.457567
1,Africa Eastern and Southern,AFE,Adults (ages 15+) and children (0-14 years) li...,SH.HIV.TOTL,1960,NaN
2,Africa Eastern and Southern,AFE,Adults (ages 15+) and children (ages 0-14) new...,SH.HIV.INCD.TL,1960,NaN
3,Africa Eastern and Southern,AFE,Adults (ages 15+) living with HIV,SH.DYN.AIDS,1960,NaN
4,Africa Eastern and Southern,AFE,Adults (ages 15-49) newly infected with HIV,SH.HIV.INCD,1960,NaN


In [8]:
#una máscara booleana que obtiene las filas que contengan el strong 'gdp'
mask = df_hnp['Indicator Name'].str.contains('gdp', flags=re.IGNORECASE)

In [9]:
#Filtro el dataframe por las filas que complan la condición, y hago un copy para preservar los datos originales
df_hnp = df_hnp[mask].copy()

In [10]:
#encontrar valores únicos de indicator con el string clave 'gdp'
df_hnp['Indicator Name'].unique()

array(['Capital health expenditure (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Domestic general government health expenditure (% of GDP)',
       'Public spending on education, total (% of GDP)'], dtype=object)

In [11]:
df_co2.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [12]:
#filtro por columnas los string que quiero encontrar en el Dataframe
df_co2 = df_co2.filter(regex='gdp|iso_code|year|country').copy()


In [13]:
df_co2.head()

,iso_code,country,year,co2_per_gdp,consumption_co2_per_gdp,gdp,energy_per_gdp
0,AFG,Afghanistan,1949,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.009,NaN,9.421400e+09,NaN
2,AFG,Afghanistan,1951,0.010,NaN,9.692280e+09,NaN
3,AFG,Afghanistan,1952,0.009,NaN,1.001732e+10,NaN
4,AFG,Afghanistan,1953,0.010,NaN,1.063052e+10,NaN


## Merge the data

In [14]:
#convierto las columnas en filas, poniendo el foco en las que me interesan para el merge
df_co2 = df_co2.melt(id_vars=['iso_code', 'country', 'year'])

In [15]:
df_co2.head()

,iso_code,country,year,variable,value
0,AFG,Afghanistan,1949,co2_per_gdp,NaN
1,AFG,Afghanistan,1950,co2_per_gdp,0.009
2,AFG,Afghanistan,1951,co2_per_gdp,0.010
3,AFG,Afghanistan,1952,co2_per_gdp,0.009
4,AFG,Afghanistan,1953,co2_per_gdp,0.010


In [16]:
#renombro la columna variable
df_co2 = df_co2.rename({"variable": 'indicator'}, axis=1)

In [17]:
#elimino columna que me sobra
df_hnp = df_hnp.drop(columns='Indicator Code')

In [18]:
# asigno una lista de nuevos nombres de columna para mejorar la legibilidad del código 
df_hnp.columns = ['country', 'iso_code', 'indicator', 'year', 'value']

In [19]:
df_hnp.head()

,country,iso_code,indicator,year,value
68,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,NaN
98,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,NaN
108,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,NaN
385,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,NaN
511,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,NaN


In [20]:
#combino mis dos dataframes añadiendo al segundo dataf las filas del primero
df = pd.concat([df_hnp, df_co2])

In [21]:
df.to_csv('data.csv', index=False)

In [22]:
df.head()

,country,iso_code,indicator,year,value
68,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,NaN
98,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,NaN
108,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,NaN
385,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,NaN
511,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,NaN


## External sources

### API

In [23]:
#extraigo los datos de la api
df_world_bank = pd.read_xml('https://api.worldbank.org/v2/source/75/indicators')

In [24]:
df_world_bank.head()

,id,{http://www.worldbank.org}name,{http://www.worldbank.org}unit,{http://www.worldbank.org}source,{http://www.worldbank.org}sourceNote,{http://www.worldbank.org}sourceOrganization,{http://www.worldbank.org}topics
0,AG.LND.AGRI.ZS,Agricultural land (% of land area),NaN,"Environment, Social and Governance (ESG) Data",Agricultural land refers to the share of land ...,"Food and Agriculture Organization, electronic ...",NaN
1,AG.LND.FRST.ZS,Forest area (% of land area),NaN,"Environment, Social and Governance (ESG) Data",Forest area is land under natural or planted s...,"Food and Agriculture Organization, electronic ...",NaN
2,AG.PRD.FOOD.XD,Food production index (2014-2016 = 100),NaN,"Environment, Social and Governance (ESG) Data",Food production index covers food crops that a...,"Food and Agriculture Organization, electronic ...",NaN
3,CC.EST,Control of Corruption: Estimate,NaN,"Environment, Social and Governance (ESG) Data",Control of Corruption captures perceptions of ...,"Detailed documentation of the WGI, interactive...",NaN
4,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...,NaN,"Environment, Social and Governance (ESG) Data",Access to clean fuels and technologies for coo...,WHO Global Health Observatory (https://www.wh...,NaN


In [25]:
#relizo solicitud a la api y la parseo con el método json y la almaceno
res = requests.get('https://api.worldbank.org/v2/country/all/indicator/AG.LND.FRST.ZS?format=json&date=1960:2020')

data = res.json()

In [26]:
list_df = []

In [27]:
#se está realizando un bucle "for" para solicitar y almacenar los datos de la API de Banco Mundial
#se ejecuta para un número de páginas igual a "data[0]['pages']+1
#Para cada página, se realiza una solicitud a la API con el método "get

for page in tqdm(range(1, data[0]['pages']+1)):
    res = requests.get(f'https://api.worldbank.org/v2/country/all/indicator/AG.LND.FRST.ZS?format=json&date=1960:2020&page={page}')
    data = res.json()
    df_data = pd.DataFrame(data[1:][0])
    list_df.append(df_data)

100%|██████████| 325/325 [06:26<00:00,  1.19s/it]


In [28]:
#concatena todos los datos en un mismo dataframe
df_api = pd.concat(list_df)

In [29]:
df_api.head()

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,30.174186,,,1
1,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,30.391558,,,1
2,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,30.611444,,,1
3,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,30.824248,,,1
4,"{'id': 'AG.LND.FRST.ZS', 'value': 'Forest area...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,31.039613,,,1


In [30]:
#selecciono las columnas que me interesan
df_api = df_api[['countryiso3code', 'date', 'value']].copy()

In [31]:
#extrae el nombre del indicador de los datos.
indicator_name = list_df[0].loc[0, 'indicator']['value']
indicator_name

'Forest area (% of land area)'

In [32]:
#reasingno el nombre de la columna indicator
df_api['indicator'] = indicator_name
indicator_name

'Forest area (% of land area)'

In [33]:
df_api.head()

,countryiso3code,date,value,indicator
0,AFE,2020,30.174186,Forest area (% of land area)
1,AFE,2019,30.391558,Forest area (% of land area)
2,AFE,2018,30.611444,Forest area (% of land area)
3,AFE,2017,30.824248,Forest area (% of land area)
4,AFE,2016,31.039613,Forest area (% of land area)


In [34]:
dic = {}

In [35]:
#crea un diccionario "dic" con claves que son códigos de país ISO y valores que son nombres de países.
for i,j in df[['country', 'iso_code']].value_counts().index:
    dic[j] = i

In [36]:
dic

{'GBR': 'United Kingdom',
 'CAN': 'Canada',
 'DEU': 'Germany',
 'USA': 'United States',
 'POL': 'Poland',
 'BEL': 'Belgium',
 'FRA': 'France',
 'AUT': 'Austria',
 'OWID_WRL': 'World',
 'NOR': 'Norway',
 'KAZ': 'Kazakhstan',
 'BLR': 'Belarus',
 'ESP': 'Spain',
 'LVA': 'Latvia',
 'LTU': 'Lithuania',
 'AZE': 'Azerbaijan',
 'ARM': 'Armenia',
 'MDA': 'Moldova',
 'TJK': 'Tajikistan',
 'GEO': 'Georgia',
 'TKM': 'Turkmenistan',
 'UKR': 'Ukraine',
 'UZB': 'Uzbekistan',
 'HUN': 'Hungary',
 'EST': 'Estonia',
 'SWE': 'Sweden',
 'DNK': 'Denmark',
 'NLD': 'Netherlands',
 'IRL': 'Ireland',
 'CHE': 'Switzerland',
 'ROU': 'Romania',
 'IND': 'India',
 'ITA': 'Italy',
 'FIN': 'Finland',
 'CZE': 'Czechia',
 'AUS': 'Australia',
 'GRC': 'Greece',
 'JPN': 'Japan',
 'PRT': 'Portugal',
 'NZL': 'New Zealand',
 'BGR': 'Bulgaria',
 'ZAF': 'South Africa',
 'PER': 'Peru',
 'BIH': 'Bosnia and Herzegovina',
 'HRV': 'Croatia',
 'MKD': 'North Macedonia',
 'MNE': 'Montenegro',
 'SVN': 'Slovenia',
 'SRB': 'Serbia',
 'ARG

In [37]:
#inserto una nueva columna "country" en el DataFrame "df_api".
#reemplazo los valores en la columna "countryiso3code" con sus correspondientes nombres de país utilizando el diccionario "dic".Toma un diccionario como argumento y utiliza los valores en este diccionario para reemplazar los valores en la columna especificada.
df_api.insert(0, 'country', df_api.countryiso3code.replace(dic))

In [38]:
#renombra las columnas "countryiso3code" y "date" en el DataFrame "df_api"
df_api = df_api.rename({'countryiso3code': 'iso_code', 'date': 'year'}, axis=1)

In [39]:
#uno los dos dataframes
df = pd.concat([df, df_api]).reset_index(drop=True)

In [40]:
df.head()

,country,iso_code,indicator,year,value
0,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,NaN
1,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,NaN
2,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,NaN
3,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,NaN
4,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,NaN


In [41]:
df['indicator'].unique()


array(['Capital health expenditure (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Domestic general government health expenditure (% of GDP)',
       'Public spending on education, total (% of GDP)', 'co2_per_gdp',
       'consumption_co2_per_gdp', 'gdp', 'energy_per_gdp',
       'Forest area (% of land area)'], dtype=object)

In [42]:
df_api.head()

,country,iso_code,year,value,indicator
0,Africa Eastern and Southern,AFE,2020,30.174186,Forest area (% of land area)
1,Africa Eastern and Southern,AFE,2019,30.391558,Forest area (% of land area)
2,Africa Eastern and Southern,AFE,2018,30.611444,Forest area (% of land area)
3,Africa Eastern and Southern,AFE,2017,30.824248,Forest area (% of land area)
4,Africa Eastern and Southern,AFE,2016,31.039613,Forest area (% of land area)


In [43]:
df[['country', 'iso_code']]

,country,iso_code
0,Africa Eastern and Southern,AFE
1,Africa Eastern and Southern,AFE
2,Africa Eastern and Southern,AFE
3,Africa Eastern and Southern,AFE
4,Africa Western and Central,AFW
...,...,...
183005,Zimbabwe,ZWE
183006,Zimbabwe,ZWE
183007,Zimbabwe,ZWE
183008,Zimbabwe,ZWE


In [44]:
#lee una tabla HTML y almacena los datos en el DataFrame "df_iso".
df_iso = pd.read_html('https://cloford.com/resources/codes/index.htm')[3]

In [45]:
#crea una copia del DataFrame "df_iso" que solo incluye las columnas "Continent", "ISO (3)" y "Country".
df_iso = df_iso[['Continent', 'ISO (3)', 'Country']].copy()

In [46]:
#renombra para poder unir con el dataframe
df_iso.columns=['continent', 'iso_code', 'country']

In [47]:
#lo paso a csv
df.to_csv('data_all_indicators.csv', index=False)

In [59]:
#df.to_csv(r"C:\Users\Usuario\Desktop\Proyecto final\Indicators.csv", index=False)


In [48]:
#lo paso a csv
df_iso.to_csv('data_iso.csv', index=False)

In [58]:
#df_iso.to_csv(r"C:\Users\Usuario\Desktop\Proyecto final\ISO.csv", index=False)


In [49]:
df['year'] = df['year'].astype(int)

In [50]:
df.head()

,country,iso_code,indicator,year,value
0,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,NaN
1,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,NaN
2,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,NaN
3,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,NaN
4,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,NaN


# Ponerlo en SQL

In [51]:
#conecto a una base de datos SQLite.

con = sqlite3.connect('database.db')

In [52]:
#guardando "df_iso" en una tabla de la base de datos SQLite "iso"
df_iso.to_sql('iso', con)

ValueError: Table 'iso' already exists.

In [ ]:
#guardando "df" en una tabla de la base de datos SQLite "iso"

df.to_sql('indicator_data', con)

In [ ]:
#Esta consulta hace una combinación de dos tablas: "indicator_data" y "iso"
query = '''
SELECT *
FROM indicator_data
INNER JOIN iso
ON indicator_data.iso_code = iso.iso_code
'''

In [ ]:
pd.read_sql(sql=query, con=con).head()

In [ ]:
fig = px.line(data_frame=df, x='year', y='value', facet_col='indicator', color='iso_code')

In [ ]:
fig.update_yaxes(matches=None)
fig.update_xaxes(tickangle=45)
fig.for_each_annotation(lambda annotation: annotation.update(textangle=22))


In [3]:
dff

NameError: name 'dff' is not defined

In [113]:
dff = df.pivot_table(index=['country', 'iso_code', 'year'], columns='indicator', values='value')

In [114]:
dff.reset_index()

indicator,country,iso_code,year,Capital health expenditure (% of GDP),Current health expenditure (% of GDP),Domestic general government health expenditure (% of GDP),Forest area (% of land area),"Public spending on education, total (% of GDP)",co2_per_gdp,consumption_co2_per_gdp,energy_per_gdp,gdp
0,,,1960,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
1,,,1961,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,,,1962,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
3,,,1963,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
4,,,1964,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27990,Zimbabwe,ZWE,2017,0.073545,7.469752,1.794181,45.451183,5.81878,0.437,0.512,0.0,2.194784e+10
27991,Zimbabwe,ZWE,2018,0.271347,8.680062,2.782935,45.332093,3.86611,0.522,0.580,0.0,2.271535e+10
27992,Zimbabwe,ZWE,2019,0.000000,7.703565,1.358270,45.213002,0.00000,0.000,0.000,0.0,0.000000e+00
27993,Zimbabwe,ZWE,2020,0.000000,0.000000,0.000000,45.093912,0.00000,0.000,0.000,0.0,0.000000e+00


In [116]:
dff.columns

Index(['Capital health expenditure (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Domestic general government health expenditure (% of GDP)',
       'Forest area (% of land area)',
       'Public spending on education, total (% of GDP)', 'co2_per_gdp',
       'consumption_co2_per_gdp', 'energy_per_gdp', 'gdp'],
      dtype='object', name='indicator')

In [108]:
#dff.to_csv(r"C:\Users\Usuario\Desktop\Proyecto final\ETL1.csv", index=False)


In [76]:
df_api.head()

,country,iso_code,year,value,indicator
0,Africa Eastern and Southern,AFE,2020,30.174186,Forest area (% of land area)
1,Africa Eastern and Southern,AFE,2019,30.391558,Forest area (% of land area)
2,Africa Eastern and Southern,AFE,2018,30.611444,Forest area (% of land area)
3,Africa Eastern and Southern,AFE,2017,30.824248,Forest area (% of land area)
4,Africa Eastern and Southern,AFE,2016,31.039613,Forest area (% of land area)


In [77]:
df_co2.head()

,iso_code,country,year,indicator,value
0,AFG,Afghanistan,1949,co2_per_gdp,NaN
1,AFG,Afghanistan,1950,co2_per_gdp,0.009
2,AFG,Afghanistan,1951,co2_per_gdp,0.010
3,AFG,Afghanistan,1952,co2_per_gdp,0.009
4,AFG,Afghanistan,1953,co2_per_gdp,0.010


In [78]:
df_hnp.head()

,country,iso_code,indicator,year,value
68,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,NaN
98,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,NaN
108,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,NaN
385,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,NaN
511,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,NaN


---

In [79]:
df_iso.head()

,continent,iso_code,country
0,Asia,AFG,Afghanistan
1,Europe,ALB,Albania
2,Africa,DZA,Algeria
3,Oceania,ASM,American Samoa
4,Europe,AND,Andorra


In [139]:
df_country = df.loc[df['country'].isin(['Spain','Morocco','Brazil','Canada','Mexico','Australia','Pakistan', 'India', 'Russia', 'Chile', 'Germany', 'Vietnam', 'China', 'Thailand', 'Egypt', 'Italy', 'Finland', 'United States'])]
df_coun_year = df_country[(df_country['year'] > 1990) & (df_country['year'] < 2018)]

In [138]:
df['country'].unique()

array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle in

In [140]:
df_coun_year['value'] = df['value'].fillna(0)


C:\Users\Usuario\AppData\Local\Temp/ipykernel_9056/923738176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coun_year['value'] = df['value'].fillna(0)


In [141]:
#df_coun_year.to_csv(r"C:\Users\Usuario\Desktop\Proyecto final\Data_visual.csv", index=False)

C:\Users\Usuario\AppData\Local\Temp/ipykernel_9056/3905340741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_coun_year['value'] = df['value'].fillna(0)


,country,iso_code,indicator,year,value
0,Africa Eastern and Southern,AFE,Capital health expenditure (% of GDP),1960,0.0
1,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),1960,0.0
2,Africa Eastern and Southern,AFE,Domestic general government health expenditure...,1960,0.0
3,Africa Eastern and Southern,AFE,"Public spending on education, total (% of GDP)",1960,0.0
4,Africa Western and Central,AFW,Capital health expenditure (% of GDP),1960,0.0
...,...,...,...,...,...
183005,Zimbabwe,ZWE,Forest area (% of land area),1964,0.0
183006,Zimbabwe,ZWE,Forest area (% of land area),1963,0.0
183007,Zimbabwe,ZWE,Forest area (% of land area),1962,0.0
183008,Zimbabwe,ZWE,Forest area (% of land area),1961,0.0


In [189]:
df_sea = pd.read_csv(r'..data\seaice.csv')

In [192]:
df_sea = df_sea[df_sea['Year'] > 1990]

In [193]:
df_sea.columns

Index(['Year', ' Month', ' Day', '     Extent', '    Missing', ' Source Data',
       'hemisphere'],
      dtype='object')

In [194]:
df_sea = df_sea.drop(columns=[' Month', ' Day', '    Missing', ' Source Data'])

In [195]:
df_sea

,Year,Extent,hemisphere
2799,1991,13.634,north
2800,1991,13.831,north
2801,1991,13.847,north
2802,1991,13.858,north
2803,1991,13.872,north
...,...,...,...
26349,2019,10.085,south
26350,2019,10.078,south
26351,2019,10.219,south
26352,2019,10.363,south


In [217]:
df_temp = pd.read_csv(r"..data\GlobalLandTemperaturesByCountry.csv")

In [221]:
df_temp = df_temp[df_temp['dt'] > '1990-01-01']


In [223]:
df_temp['dt'] = pd.to_datetime(df_temp['dt'])

C:\Users\Usuario\AppData\Local\Temp/ipykernel_9056/710794200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['dt'] = pd.to_datetime(df_temp['dt'])


In [230]:
df_temp = df_temp.groupby([df_temp['dt'].dt.year, 'Country'])['AverageTemperature'].mean().reset_index()


In [232]:
df_temp = df_temp.rename(columns={'dt': 'year'})
df_temp = df_temp.rename(columns={'Country': 'country'})

In [2]:
#df_sea.to_csv(r"..data\sea.csv", index=False)
#df_temp.to_csv(r"..data\temp.csv", index=False)

NameError: name 'df_sea' is not defined

In [235]:
df_temp

,year,country,AverageTemperature
0,1990,Afghanistan,16.214727
1,1990,Africa,25.002091
2,1990,Albania,14.483091
3,1990,Algeria,25.087818
4,1990,American Samoa,27.152000
...,...,...,...
5827,2013,Western Sahara,23.744250
5828,2013,Yemen,28.129750
5829,2013,Zambia,21.196000
5830,2013,Zimbabwe,20.710750
